In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
import sys
sys.path.append(os.path.abspath(".."))
import torch

from datasets import PressingSequenceDataset, SoccerMapInputDataset

2025-06-07 07:47:35.320810: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-07 07:47:35.323190: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-07 07:47:35.352304: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-07 07:47:35.352336: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-07 07:47:35.352359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
data_path = "/data/MHL/pressing-intensity"

with open(f"{data_path}/train_dataset.pkl", "rb") as f:
    train_dataset = pickle.load(f)

with open(f"{data_path}/test_dataset.pkl", "rb") as f:
    test_dataset = pickle.load(f)

len(train_dataset), len(test_dataset)

(14241, 1721)

In [3]:
sample = train_dataset[0]
sample.keys()

dict_keys(['features', 'pressing_intensity', 'label', 'presser_id', 'agent_order'])

In [4]:
print(f"Features : {sample['features'].shape}")
print(f"Pressing Intensity : {sample['pressing_intensity'].shape}")
print(f"Labels : {sample['label']}")
print(f"Presser ID : {sample['presser_id']}")
print(f"Players Order : {sample['agent_order']}")

Features : torch.Size([150, 23, 8])
Pressing Intensity : torch.Size([150, 11, 11])
Labels : 0
Presser ID : 77414
Players Order : ['188178', '250079', '250101', '250102', '500133', '500140', '500141', '500142', '62365', '62386', '77414', '187259', '343587', '408792', '500113', '500115', '500116', '500117', '500118', '500121', '500502', '83615', 'ball']


In [5]:
x_tensor = sample['features']
press_intensity = sample['pressing_intensity']
y_tensor = sample['label']
x_tensor.shape, press_intensity.shape, y_tensor

(torch.Size([150, 23, 8]), torch.Size([150, 11, 11]), tensor(0))

# 1. XGBoost

In [ ]:
num_seq = 150
num_agents = 11
use_pressing_intensity = False
selected_features_idx = [1, 2, 4, 7]

train_features = []
train_labels = []
for i in range(len(train_dataset)):
    sample = train_dataset[i]
    x_tensor = sample['features'][..., selected_features_idx][-90:]
    press_intensity = sample['pressing_intensity'][-90:]
    y_tensor = sample['label']
    
    # Flatten the sequence data: shape (sequence_length, num_features) -> (sequence_length*num_features,)
    feature_vector = x_tensor.flatten().numpy()
    if use_pressing_intensity:
        if press_intensity.shape[1] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], num_agents-press_intensity.shape[1], press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=1)
            
        if press_intensity.shape[2] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], press_intensity.shape[1], num_agents-press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=2)
        
        press_vector = press_intensity.flatten().numpy()
        # Concatenate the flattened sequence data with the pressing intensity
        feature_vector = np.concatenate((feature_vector, press_vector))
    train_features.append(feature_vector)
    train_labels.append(y_tensor.item())


test_features = []
test_labels = []
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    x_tensor = sample['features'][..., selected_features_idx][-90:]
    press_intensity = sample['pressing_intensity'][-90:]
    y_tensor = sample['label']
    
    # Flatten the sequence data: shape (sequence_length, num_features) -> (sequence_length*num_features,)
    feature_vector = x_tensor.flatten().numpy()
    
    if use_pressing_intensity:
        if press_intensity.shape[1] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], num_agents-press_intensity.shape[1], press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=1)
        if press_intensity.shape[2] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], press_intensity.shape[1], num_agents-press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=2)
        
        press_vector = press_intensity.flatten().numpy()
        # Concatenate the flattened sequence data with the pressing intensity
        feature_vector = np.concatenate((feature_vector, press_vector))
    test_features.append(feature_vector)
    test_labels.append(y_tensor.item())

X_train = np.array(train_features)
y_train = np.array(train_labels)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_test = np.array(test_features)
y_test = np.array(test_labels)

: 

In [8]:
def print_dataset_distribution(y_train, y_val, y_test):
    def _print_split(name, labels):
        total = len(labels)
        unique, counts = np.unique(labels, return_counts=True)
        print(f"{name} Set:")
        print(f"  Total samples: {total}")
        for label, count in zip(unique, counts):
            percent = (count / total) * 100
            print(f"    Label {label}: {count:>5} samples ({percent:5.2f}%)")
        print("-" * 40)

    print("\n📊 Dataset Distribution Summary")
    print("=" * 40)
    _print_split("Train", y_train)
    _print_split("Validation", y_val)
    _print_split("Test", y_test)

print_dataset_distribution(y_train, y_val, y_test)


📊 Dataset Distribution Summary
Train Set:
  Total samples: 11392
    Label 0:  9147 samples (80.29%)
    Label 1:  2245 samples (19.71%)
----------------------------------------
Validation Set:
  Total samples: 2849
    Label 0:  2286 samples (80.24%)
    Label 1:   563 samples (19.76%)
----------------------------------------
Test Set:
  Total samples: 1721
    Label 0:  1359 samples (78.97%)
    Label 1:   362 samples (21.03%)
----------------------------------------


In [9]:
# Create XGBoost DMatrix objects for train and test sets
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

# Set XGBoost training parameters
params = {
    'objective': 'binary:logistic',  # binary classification
    'eval_metric': 'auc',            # evaluation metric: AUC
    'max_depth': 6,                  # maximum depth of trees
    'eta': 0.1,                      # learning rate
    'seed': 42
}

# Specify the watchlist to evaluate performance on training and test sets during training
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
num_rounds = 100

In [10]:
# Train XGBoost model with early stopping on the evaluation set
bst = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=10)

/home/work/miniconda/envs/mhl_py311/lib/python3.11/site-packages/xgboost/core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-auc:0.72404	eval-auc:0.59755
[1]	train-auc:0.77449	eval-auc:0.61684
[2]	train-auc:0.80143	eval-auc:0.61842
[3]	train-auc:0.82563	eval-auc:0.62081
[4]	train-auc:0.83993	eval-auc:0.62739
[5]	train-auc:0.85611	eval-auc:0.63009
[6]	train-auc:0.86313	eval-auc:0.63397
[7]	train-auc:0.87409	eval-auc:0.63416
[8]	train-auc:0.88935	eval-auc:0.63792
[9]	train-auc:0.89944	eval-auc:0.63648
[10]	train-auc:0.90767	eval-auc:0.63826
[11]	train-auc:0.91626	eval-auc:0.63678
[12]	train-auc:0.92196	eval-auc:0.63293
[13]	train-auc:0.92891	eval-auc:0.63344
[14]	train-auc:0.93481	eval-auc:0.63360
[15]	train-auc:0.93753	eval-auc:0.63349
[16]	train-auc:0.94254	eval-auc:0.63322
[17]	train-auc:0.94739	eval-auc:0.63347
[18]	train-auc:0.94938	eval-auc:0.63573
[19]	train-auc:0.95371	eval-auc:0.63360
[20]	train-auc:0.95596	eval-auc:0.63550


In [ ]:
# W/O Pressing Intensity
# Get predictions on the test set
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_val, y_pred_label)
auc = roc_auc_score(y_val, y_pred)

print("Test Accuracy: {:.4f}".format(accuracy))
print("Test AUC: {:.4f}".format(auc))

Test Accuracy: 0.8027
Test AUC: 0.6491


: 

In [11]:
# W/O Pressing Intensity
# Get predictions on the test set
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_val, y_pred_label)
auc = roc_auc_score(y_val, y_pred)

print("Test Accuracy: {:.4f}".format(accuracy))
print("Test AUC: {:.4f}".format(auc))

Test Accuracy: 0.7989
Test AUC: 0.6465


# 2. Deep Learning

In [5]:
from model import ToSoccerMapTensor

SoccerMapTensor = ToSoccerMapTensor()

In [57]:
i = 0
sample = train_dataset[i]
x_tensor = sample['features']
press_intensity = sample['pressing_intensity']
y_tensor = sample['label']
presser_id = sample['presser_id']
agents_order = sample['agent_order']
presser_idx = agents_order.index(presser_id)
home_team_ids = agents_order[:11]
away_team_ids = agents_order[11:22]


In [45]:
x_feat = x_tensor[-1, ...]
x_feat.shape

torch.Size([23, 8])

In [70]:
x_tensor.shape

torch.Size([125, 23, 8])

In [46]:
y_bins, x_bins = (68, 104)
num_features = 13
nb_prev_actions = 1
matrix = np.zeros((num_features * nb_prev_actions, y_bins, x_bins)) # 13, 68, 104
# Location of the goal
goal_coo = torch.tensor([[52.5, 0]])
matrix.shape

(13, 68, 104)

In [47]:
# def _get_cell_indexes(self, x: torch.Tensor, y: torch.Tensor):
def _get_cell_indexes(x: torch.Tensor, y: torch.Tensor):
    # Shift the origin from the pitch center to the bottom-left corner:
    #   x: [-52.5, 52.5] -> [0, 105]
    #   y: [-34.0,  34.0] -> [0,  68]
    x_shifted = x + 52.5
    y_shifted = y + 34.0

    # Normalize into [0, 1]
    x_norm = x_shifted / 105.0
    y_norm = y_shifted /  68.0

    # Scale to bin coordinates
    # x_cont = x_norm * self.x_bins
    # y_cont = y_norm * self.y_bins
    x_cont = x_norm * x_bins
    y_cont = y_norm * y_bins
    
    # Clamp into valid index range [0, bins-1]
    # x_clamped = torch.clamp(x_cont, min=0, max=self.x_bins - 1)
    # y_clamped = torch.clamp(y_cont, min=0, max=self.y_bins - 1)
    x_clamped = torch.clamp(x_cont, min=0, max=x_bins - 1)
    y_clamped = torch.clamp(y_cont, min=0, max=y_bins - 1)

    # Convert to integer bin indices
    x_bin = x_clamped.to(torch.int64).to(torch.uint8)
    y_bin = y_clamped.to(torch.int64).to(torch.uint8)

    return x_bin, y_bin

In [50]:
i = 0
home_indices = slice(0, 11)
away_indices = slice(11, 22)

if presser_id in home_team_ids:
    pressing_indices = home_indices
    pressed_indices = away_indices
    x_bin_pressing, y_bin_pressing = _get_cell_indexes(x_feat[pressing_indices, :1], x_feat[pressing_indices, 1:2])
    x_bin_pressed, y_bin_pressed = _get_cell_indexes(x_feat[pressed_indices, :1], x_feat[pressed_indices, 1:2])
elif presser_id in away_team_ids:
    pressed_indices = home_indices
    pressing_indices = away_indices
    x_bin_pressed, y_bin_pressed= _get_cell_indexes(x_feat[pressed_indices, :1], x_feat[pressed_indices, 1:2])
    x_bin_pressing, y_bin_pressing = _get_cell_indexes(x_feat[pressing_indices, :1], x_feat[pressing_indices, 1:2])
else:
    raise ValueError(f"Invalid presser_id: {presser_id}")

# Ch 1: Locations of pressing teams
matrix[0 + i * num_features, y_bin_pressing, x_bin_pressing] = 1

# Ch 2: vx of pressing teams
matrix[1 + i * num_features, y_bin_pressing, x_bin_pressing] = x_feat[pressing_indices, 2:3]

# Ch 3: vy of pressing teams
matrix[2 + i * num_features, y_bin_pressing, x_bin_pressing] = x_feat[pressing_indices, 3:4]

# CH 4: Locations of pressed teams
matrix[3 + i * num_features, y_bin_pressed, x_bin_pressed] = 1

# Ch 5: vx of pressed teams
matrix[4 + i * num_features, y_bin_pressed, x_bin_pressed] = x_feat[pressed_indices, 2:3]

# Ch 6: vy of pressed teams
matrix[5 + i * num_features, y_bin_pressed, x_bin_pressed] = x_feat[pressed_indices, 3:4]

# CH 7: Distance to ball
y_coords = torch.arange(0.5, y_bins, device=x_feat.device) # Shape: [y_bins]
x_coords = torch.arange(0.5, x_bins, device=x_feat.device)

# Create 2D grid coordinate tensors
# 'ij' indexing: yy changes along dim 0, xx changes along dim 1
yy, xx = torch.meshgrid(y_coords, x_coords, indexing='ij')
# yy shape: [y_bins, x_bins], xx shape: [y_bins, x_bins]

x0_ball, y0_ball = _get_cell_indexes(x_feat[22, :1], x_feat[22, 1:2])
x0_ball_center = x0_ball.float() + 0.5
y0_ball_center = x0_ball.float() + 0.5

ball_distance = torch.sqrt((xx - x0_ball_center)**2 + (yy - y0_ball_center)**2)
matrix[6 + i * num_features, : , :] = ball_distance.numpy()

# CH 8: Distance to goal
x0_goal, y0_goal = _get_cell_indexes(goal_coo[:, 0], goal_coo[:, 1])
x0_goal_center = x0_goal.float() + 0.5
y0_goal_center = y0_goal.float() + 0.5

goal_distance = torch.sqrt((xx - x0_goal_center)**2 + (yy - y0_goal_center)**2)
matrix[7 + i * num_features, : , :] = goal_distance.numpy()


In [51]:
# CH 9: Cosine of the angle between the ball and goal
coords = torch.stack([xx, yy], dim=-1) # Shape: [H, W, 2]
goal_coo_bin = torch.tensor([x0_goal_center, y0_goal_center], dtype=torch.float32, device=x_feat.device) # Shape: [2]
ball_coo_bin = torch.tensor([x0_ball_center, y0_ball_center], dtype=torch.float32, device=x_feat.device) # Shape: [2]
a = goal_coo_bin - coords # Shape: [H, W, 2], vector from cell (i,j) to goal
b = ball_coo_bin - coords # Shape: [H, W, 2], vector from cell (i,j) to ball

norm_a = torch.linalg.norm(a, dim=-1) # Shape: [H, W]
norm_b = torch.linalg.norm(b, dim=-1) # Shape: [H, W]
denominator = norm_a * norm_b + 1e-8 # Add epsilon here
cosine_angle = torch.sum(a * b, dim=-1) / (norm_a + norm_b + 1e-8)
cosine_angle = torch.clamp(cosine_angle, min=-1.0, max=1.0) # Shape: [H, W]
matrix[8 + i * num_features, : , :] = cosine_angle.numpy()

In [67]:
# CH 10: Sine of the angle between the ball and goal
sine_angle = (a[..., 0] * b[..., 1] - a[..., 1] * b[..., 0]) / (norm_a + norm_b + 1e-8)
sine_angle = torch.clamp(sine_angle, min=-1.0, max=1.0)
matrix[9 + i * num_features, : , :] = sine_angle.numpy()

In [53]:
# CH 11: Angle (in radians) to the goal location
vector_y = y0_goal_center - coords[..., 1] # Shape: [H, W]
vector_x = x0_goal_center - coords[..., 0] # Shape: [H, W]
angle_rad = torch.abs(torch.arctan2(vector_y, vector_x))
matrix[10 + i * num_features, : , :] = angle_rad.numpy()

In [ ]:
# CH 12, 13: Cosine, sine angle to the teammates from pressers
teammates_idx = [i for i in range(pressing_indices.start, pressing_indices.stop) if i != presser_idx]

pos_presser = x_feat[presser_idx, 0:2]           # Ball carrier position [x, y], Shape: [2]
v_presser = x_feat[presser_idx, 2:4]            # Ball carrier velocity [vx, vy], Shape: [2] -> ASSUMPTION about indices 2, 3
pos_teammates = x_feat[teammates_idx, 0:2]

vec_to_teammate = pos_teammates - pos_presser # Shape: [10, 2]
norm_v_carrier = torch.linalg.norm(v_presser)        # Scalar magnitude |u|
norm_vec_to_tm = torch.linalg.norm(vec_to_teammate, dim=1) # Magnitude |v| for each teammate, Shape: [10]

cosine_angle = torch.sum(v_presser.unsqueeze(0) * vec_to_teammate, dim=1) / (norm_v_carrier * norm_vec_to_tm + 1e-8) # Shape: [10]
sine_angle = (v_presser[0] * vec_to_teammate[:, 1] - v_presser[1] * vec_to_teammate[:, 0]) / (norm_v_carrier * norm_vec_to_tm + 1e-8)   # Shape: [10]
cosine_angle = torch.clamp(cosine_angle, min=-1.0, max=1.0)
sine_angle = torch.clamp(sine_angle, min=-1.0, max=1.0)

x_bin_tm, y_bin_tm = _get_cell_indexes(pos_teammates[:, 0], pos_teammates[:, 1])
matrix[11 + i * num_features, y_bin_tm, x_bin_tm] = cosine_angle.numpy()
matrix[12 + i * num_features, y_bin_tm, x_bin_tm] = sine_angle.numpy()

In [13]:
DATA_PATH = "/data/MHL/pressing-intensity" # Path where pickled datasets are saved

train_dataset = SoccerMapInputDataset(os.path.join(DATA_PATH, "train_dataset.pkl"))

Loading dataset from /data/MHL/pressing-intensity/train_dataset.pkl...


In [15]:
X_tensor, y = train_dataset[0]
X_tensor.shape

torch.Size([1625, 68, 104])

In [4]:
pickled_dataset_path = os.path.join(DATA_PATH, "train_dataset.pkl")

with open(pickled_dataset_path, "rb") as f:
    # Load the dictionary saved by PressingSequenceDataset
    loaded_data = pickle.load(f)

In [6]:
num_samples = len(loaded_data)

In [16]:
from torch.utils.data import DataLoader
# Hyperparameters
BATCH_SIZE = 32 # Adjust based on GPU memory
LEARNING_RATE = 1e-4
EPOCHS = 20 # Adjust as needed
NUM_WORKERS = 4 # Adjust based on system cores
VAL_SPLIT_RATIO = 0.2 # Use 15% of training data for validation


train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True if NUM_WORKERS > 0 else False,
        # collate_fn=collate_fn_skip_none # Use this if __getitem__ can return None
    )


In [17]:
train_data = next(iter(train_loader))

In [18]:
len(train_data)

2

In [20]:
from model import PytorchSoccerMapModel

In [32]:
CRITERION = 'bce' # 'bce' or 'focal'
CRITERION_PARAMS = {} # Add params like alpha, gamma for FocalLoss if needed

optimizer_params = {
"optimizer_params": {
    "lr": 1e-4,
    "weight_decay": 1e-5
}
}
model_config = {
"in_channels": 125 * 13
}

model = PytorchSoccerMapModel(model_config=model_config, optimizer_params=optimizer_params)
model = model.to("cuda")


In [33]:
x, y = train_data
x = x.to("cuda")
y = y.to("cuda")

In [34]:
out = model(x)

# 3. exPress

In [29]:
from torch_geometric.loader import DataLoader as PyGDataLoader
from datasets import exPressInputDataset

In [30]:
data_path = "/data/MHL/pressing-intensity"
full_train_dataset = exPressInputDataset(f"{data_path}/train_dataset.pkl")

Loading dataset from /data/MHL/pressing-intensity/train_dataset.pkl...


In [33]:
full_train_dataset[0]['features'].max(), full_train_dataset[0]['features'].min() 

(tensor(80.7753), tensor(-56.0759))

In [ ]:
features_tensor = full_train_dataset[0]['features']

min_in_tensor = torch.min(torch.min(features_tensor, dim=0).values, dim=0).values
max_in_tensor = torch.max(torch.max(features_tensor, dim=0).values, dim=0).values

current_min_vals = torch.minimum(current_min_vals, min_in_tensor)
current_max_vals = torch.maximum(current_max_vals, max_in_tensor)

torch.Size([23, 8])

In [38]:
full_train_dataset[i]['features'].max(), full_train_dataset[i]['features'].min()

(tensor(239.0878), tensor(-194.9801))

In [ ]:
for i in range(len(full_train_dataset)):
    full_train_dataset[i]['features'].max(), full_train_dataset[i]['features'].min()
    

In [39]:
train_loader = PyGDataLoader(
        full_train_dataset,
        batch_size=16,
        shuffle=True,
        num_workers=4
        # persistent_workers 등은 PyTorch DataLoader와 유사하게 설정 가능
    )

In [43]:
data = next(iter(train_loader))
data.keys()

dict_keys(['features', 'pressing_intensity', 'label', 'presser_id', 'agent_order'])

In [42]:
data['features'].shape, data['pressing_intensity'].shape, data['label'].shape

(torch.Size([16, 125, 23, 8]), torch.Size([16, 125, 11, 11]), torch.Size([16]))

In [48]:
B, T, A, F_in = data['features'].shape
x_frames = data['features'].view(B * T, A, F_in)

In [50]:
import torch.nn as nn
import torch.nn.functional as F
node_embed = nn.Linear(F_in, 64)
node_embed = node_embed.to("cuda")
x_frames = x_frames.to("cuda")
node_features = F.relu(node_embed(x_frames)) # [B*T, A, F_embed]

In [51]:
node_features.shape

torch.Size([2000, 23, 64])

In [52]:
node_features_flat = node_features.view(-1, node_features.shape[-1])
node_features_flat.shape

torch.Size([46000, 64])

In [53]:
batch_idx = torch.arange(B * T, device="cuda").repeat_interleave(A)

In [57]:
device="cuda"
agents_indices = torch.arange(A, device=device)
edge_index_player = torch.cartesian_prod(agents_indices, agents_indices).t()
edge_index_player = edge_index_player[:, edge_index_player[0] != edge_index_player[1]] # Remove self-loops -> [2, 22*21]


In [59]:
num_edges_per_graph = edge_index_player.shape[1]
num_edges_per_graph

506

In [63]:
edge_index_batch = edge_index_player.repeat(1, B * T) + node_offset
edge_index_batch.shape

torch.Size([2, 1012000])

In [65]:
edge_weight_batch = torch.ones(edge_index_batch.shape[1], device=device)

edge_weight_batch.shape

torch.Size([1012000])

In [66]:
node_features.shape

torch.Size([2000, 23, 64])